# Lesson 9: Algorithms -- Backtracking
---
In this lesson, we will cover the following parts:
* 9.1: Lecture Note
* 9.2: Leetcode Training (Basic)
* 9.3: Leetcode Practice (Advanced)

## 9.1 Lecture Note
### 9.1.1 What is backtracking 回溯法

Recall that recursion is a really powerful tool for designing algorithm. Generally speaking, in reality, we know how to solve some basic problems. For any other problem, the most common technique is to try to reduce these new problems to those existing basic problems we are familiar with. Recursion is a special kind of reduction that reduces the problem to one or more simpler instances of the same problem.

In earlier lectures, you are exposed to these certain recursion algorithms: merge sort, quick sort, etc. All of them have a common pattern called divide and conquer 分治法:
1. Divide the instance of the problem into several independent smaller instances.
2. Solve these smaller instances by recursion.
3. Gather all those results and combine them in a way to get the correct answer for the original instance of the problem.

Usually, some problems are hard to expressed in this divide and conquer recursion manner. In this case, we often resort to another common recursion strategy called backtracking. 该类问题特点：多阶段/多步骤决策问题。破题思路：穷举法。Basically, at the high level, it involves building a set of candidates incrementally and abandons a candidate ("backtracks") as soon as it determines that the candidate cannot possibly be a valid solution. 

The following is an example of the meaning of "building a set of candidates incrementally". Assuming right now you want to make a cake, there are 3 steps: picking a certain type of flour, cream and fruit. Let's say, right now, we have 3 types of flour, 2 types of cream and 6 types of fruit. How do we generate all the possible recipes of making cakes out of these materials?

In the simplest form, we can have something like the following:
```python
cakes = []
for flour in [flour1, flour2, flour3]:
    for cream in [cream1, cream2]:
        for fruit in [fruit1, fruit2, fruit3, fruit4, fruit5, fruit6]:
            cakes.append(make_cake(flour, cream, fruit))
```
As you can see, we first try to make a decision on the type of flour. Once that is determined, we will move on to make a decision on the type of cream, and so on. Basically, for our problem, the answer, which is a cake, is defined by this tuple (flour, cream, fruit) and we build this incrementally.

Now, let's generalize this. What if all the choices that we make during our answer generation can be compatible? For example, assuming flour 1 does not work well with cream2, flour3 does not work well with cream1, cream1 and fruit3 does not taste well together, cream2 and fruit1 or furit6 does not taste well together, how to generate all the possible tasty cakes then?

Our solution will become more complicated
```python
cakes = []
for flour in [flour1, flour2, flour3]:
    for cream in [cream1, cream2]:
        if (flour, cream) in [(flour1, cream2), (flour3, cream1)]:
            continue  # jump out of the current iteration, and continue to the next iteration
        for fruit in [fruit1, fruit2, fruit3, fruit4, fruit5, fruit6]:
            cakes.append(make_cake(flour, cream, fruit))
            if (cream, fruit) in [(cream1, fruit3), (cream2, fruit1), (cream2, fruit6)]:
                continue
            cakes.append(make_cake(flour, cream, fruit))
```

As you can see, the logic flow becomes a bit more involved. What if the number of steps to generate a single answer is big then? In this case, as you can easily imagine, it is infeasible to write all those for loops and conditions. Or What if the number of steps for each stage is not a determinant 不是一个确定的数值? We cannot use the simple for loop solution as shown in the above, but can refer to backtracking instead.

### 9.1.2 Basic algorithm template
Backtracking allow you to resolve the following problem in a tractable way: how do we systematically and exhaustively generate all the possible tuples that define our answers?

Abstractly, assuming you are trying to find out all possible sequences $x_1, x_2, x_3, \ldots, x_n$ for which a property $P_{n}(x_1, x_2, x_3, \ldots, x_n)$ holds, where each item $x_{k}$ comes from a domain $D_{k}$, in order to systematically examine all fruitful possibilities while exiting gracefully situations that have been fully explored, we can have the following meta algorithms (From Knuth's TAOCP 5B):
* B1. [Initialize] Set I = 1, and initialize the data structures needed later.
* B2. [Enter level I] (Now $P_{I-1}(x_1, x_2, x_3, \ldots, x_{I-1})$) holds. If I > n, we have a solution $x_1, x_2, x_3, \ldots, x_n$ and go to B5. Otherwise, set $x_{I} =$ min element in $D_{I}$.
* B3. [Try $x_{I}$] If $P_{I}(x_1, x_2, x_3, \ldots, x_{I})$ holds, update the data structure in order to test $P_{I+1}$, then set I = I + 1 and go to B2.
* B4. [Try another value for $x_{I}$] If $x_{I} != $ max element in $D_{I}$, set $x_{I}$ to next larger value and go to B3.
* B5. [Backtrack] Set I = I - 1. If I > 0, downdate the data structures by undoing the changes recently made in step B3, and return to B4. (Otherwise stop).

Notice in setp B3, we will only go ahead to try more of our test passes. Doing this can help us eliminate a large number of candidates as soon as possible as well as pruning the entire search space.

If we express the above in pseudo code, it will be the following:
```python
def backtracking(results, solution, current_position, N, possible_decisions):
    """
    results: store all of the feasible and compatible solutions. 最终返回结果，比如上例中的蛋糕总食谱
    solution: tuple that should contain those compatible decisions we previously made. The size of it should be == current_position (not N). 
        When the size of the solution == N, then the solution will generate a cake recipe 一个蛋糕食谱(flour, cream, fruit) as shown in the above example. Now, this is a feasible and compatible solution to the original problem. We can store this solution to the final results that we want to return.
    current_position: integer indicates the id of the step we are at right now 当前的阶段（当前是第几个阶段）. It starts at 0.
    N: integer indicates the total number of steps to build our final answer 总阶段数，比如上例中有3个阶段
    possible_decisions: a map that associates the id of a step to a collection of possible decisions we can make for that step. 当前阶段可以做的决策种类（比如说上例中第一个阶段可以选择的3种面粉，第二个阶段可以选择的2种奶油，第三个阶段的6种水果等等）   
    """
    if len(solution) == N:
        # we have successfully build one answer here
        answers.append(solution)
        ......
    else:
        for decision in possible_decisions[current_position]:
            if is_compatible(solution, decision):  # checking whether the constraints are satisfied 当前的选择和之前的决策是不是相容的。如果结果是不相容的，我们就不会去尝试后面的可能性。从这个意义上说，backtracking is a smart and efficient algorithm
                # 如果结果是相容的，我们就用当前的选择更新我们的决策
                solution.add(decision)
                # 如果结果是相容的，我们才会去尝试后面的所有的可能性
                backtracking(results, solution, current_position+1, N, possible_decisions)
                # 退出当前的选择，这样我们才能够去尝试其他的可能性
                solution.remove(decision)
```

In fact, if we model a given instance of a problem as a node and a single item $x_{k}$ (the thing we are currently trying to add to our partial result) as an edge, then all those different partially built candidates set can be formed into a tree structure as the following:
![Tree](source/lesson10_DFS_treenode.png)
Each partial cnadidate is a path from the root to a certain subtree's root. If our testing fails with a specific candidate set, for example $x_{1}$, $x_{2}$, $x_{3}$, then the whole subtree rooted at the end pooint of edge $x_{3}$ will not be explored. After this, we will try to see whether we can add another candidate, which means we will go back to the parent of a node and then try to go deeper along another path in our tree.

Thus, you can easily see the connection between backtracking and preorder tree traversal. They are basically the same: traverses this search tree recursively, from the root down, in depth-first manner.
![Tree](source/lesson10_DFS_treenode2.png)

### 9.1.3 Description of the method [Refer to Backtracking Wiki](https://en.wikipedia.org/wiki/Backtracking)
This subsection is part of supplementary materials for better understanding of backtracking.

The backtracking algorithm enumerates a set of partial candidates that, in principle, could be completed in various ways to give all the possible solutions to the given problem. The completion is done incrementally, by a sequence of candidate extension steps.

Conceptually, the partial candidates are represented as the nodes of a tree structure, the potential search tree. Each partial candidate is the parent of the candidates that differ from it by a single extension step; the leaves of the tree are the partial candidates that cannot be extended any further.

The backtracking algorithm traverses this search tree recursively, from the root down, in depth-first order. At each node c, the algorithm checks whether c can be completed to a valid solution. If it cannot, the whole sub-tree rooted at c is skipped (pruned). Otherwise, the algorithm (1) checks whether c itself is a valid solution, and if so reports it to the user; and (2) recursively enumerates all sub-trees of c. The two tests and the children of each node are defined by user-given procedures.

Therefore, the actual search tree that is traversed by the algorithm is only a part of the potential tree. The total cost of the algorithm is the number of nodes of the actual tree times the cost of obtaining and processing each node. This fact should be considered when choosing the potential search tree and implementing the pruning test.

#### Pseudocode
<font color='red' size=15> *** </font>
In order to apply backtracking to a specific class of problems, one must provide the data P for the particular instance of the problem that is to be solved, and six procedural parameters, root, reject, accept, first, next, and output. These procedures should take the instance data P as a parameter and should do the following:

root(P): return the partial candidate at the root of the search tree.
reject(P,c): return true only if the partial candidate c is not worth completing.
accept(P,c): return true if c is a solution of P, and false otherwise.
first(P,c): generate the first extension of candidate c.
next(P,s): generate the next alternative extension of a candidate, after the extension s.
output(P,c): use the solution c of P, as appropriate to the application.
The backtracking algorithm reduces the problem to the call bt(root(P)), where bt is the following recursive procedure:
```
procedure bt(c)
  if reject(P,c) then return
  if accept(P,c) then output(P,c)
  s ← first(P,c)
  while s ≠ NULL do
    bt(s)
    s ← next(P,s)
```

#### Usage considerations
The reject procedure should be a boolean-valued function that returns true only if it is certain that no possible extension of c is a valid solution for P. If the procedure cannot reach a definite conclusion, it should return false. An incorrect true result may cause the bt procedure to miss some valid solutions. The procedure may assume that reject(P,t) returned false for every ancestor t of c in the search tree.

On the other hand, the efficiency of the backtracking algorithm depends on reject returning true for candidates that are as close to the root as possible. If reject always returns false, the algorithm will still find all solutions, but it will be equivalent to a brute-force search.

The accept procedure should return true if c is a complete and valid solution for the problem instance P, and false otherwise. It may assume that the partial candidate c and all its ancestors in the tree have passed the reject test.

The general pseudo-code above does not assume that the valid solutions are always leaves of the potential search tree. In other words, it admits the possibility that a valid solution for P can be further extended to yield other valid solutions.

The first and next procedures are used by the backtracking algorithm to enumerate the children of a node c of the tree, that is, the candidates that differ from c by a single extension step. The call first(P,c) should yield the first child of c, in some order; and the call next(P,s) should return the next sibling of node s, in that order. Both functions should return a distinctive "NULL" candidate, if the requested child does not exist.

Together, the root, first, and next functions define the set of partial candidates and the potential search tree. They should be chosen so that every solution of P occurs somewhere in the tree, and no partial candidate occurs more than once. Moreover, they should admit an efficient and effective reject predicate.

#### Early stopping variants
The pseudo-code above will call output for all candidates that are a solution to the given instance P. The algorithm can be modified to stop after finding the first solution, or a specified number of solutions; or after testing a specified number of partial candidates, or after spending a given amount of CPU time.

### 9.1.4 Python Loop Control: Break, Continue, Pass, and Else [ref1](https://docs.python.org/3/tutorial/controlflow.html), [ref2](https://www.digitalocean.com/community/tutorials/how-to-use-break-continue-and-pass-statements-when-working-with-loops-in-python-3)
This subsection is part of supplementary materials for better understanding loop control in Python.

#### Break
The break statement, like in C, breaks out of the innermost enclosing for or while loop. In more details, the break statement provides you with the opportunity to exit out of (i.e., terminiate) the current loop when an external condition is triggered. You’ll put the break statement within the block of code under your loop statement, usually after a conditional if statement.

Let's look at an example that uses the break statement in a for loop:
```python
number = 0

for number in range(10):
    number = number + 1

    if number == 5:
        break    # break here

    print('Number is ' + str(number))

print('Out of loop')
```
1. In this small program, the variable number is initialized at 0. Then a for statement constructs the loop as long as the variable number is less than 10.
2. Within the for loop, the number increases incrementally by 1 with each pass because of the line number = number + 1.
3. Then, there is an if statement that presents the condition that if the variable number is equivalent to the integer 5, then the loop will break.
4. Within the loop is also a print() statement that will execute with each iteration of the for loop until the loop breaks, since it is after the break statement.
5. To see when we are out of the loop, we have included a final print() statement outside of the for loop.

When we run this code, our output will be the following:
```
Number is 1
Number is 2
Number is 3
Number is 4
Out of loop
```
This shows that once the integer number is evaluated as equivalent to 5, the loop breaks, as the program is told to do so with the break statement.

The break statement causes a program to break out of a loop.

#### Continue
The continue statement, also borrowed from C, continues with the next iteration of the loop. In more details: the continue statement gives you the option to skip over the rest part of the current iteration where an external condition is triggered, but to go on to complete the rest (next iterations) of the loop. That is, the current iteration of the loop will be disrupted, but the program will continue to execute next iterations of the loop. The continue statement will be within the block of code under the loop statement, usually after a conditional if statement.

Using the same for loop program as in the Break Statement section above, we’ll use a continue statement rather than a break statement:
```python
number = 0

for number in range(10):
    number = number + 1

    if number == 5:
        continue    # continue here

    print('Number is ' + str(number))

print('Out of loop')
```
The difference in using the continue statement rather than a break statement is that our code will continue despite the disruption when the variable number is evaluated as equivalent to 5. Let’s look at our output:
```
Number is 1
Number is 2
Number is 3
Number is 4
Number is 6
Number is 7
Number is 8
Number is 9
Number is 10
Out of loop
```
Here we see that the line Number is 5 never occurs in the output, but the loop continues after that point to print lines for the numbers 6-10 before leaving the loop.

You can use the continue statement to avoid deeply nested conditional code, or to optimize a loop by eliminating frequently occurring cases that you would like to reject. The continue statement causes a program to skip certain factors that come up within a loop, but then continue through the rest of the loop.

#### Pass
The pass statement does nothing. It can be used when a statement is required syntactically but the program requires no action. For example:
```python
while True:
    pass  # Busy-wait for keyboard interrupt (Ctrl+C)
```
This is commonly used for creating minimal classes:
```python
class MyEmptyClass:
    pass
```
Another place pass can be used is as a place-holder for a function or conditional body when you are working on new code, allowing you to keep thinking at a more abstract level. The pass is silently ignored:
```python
def initlog(*args):
    pass   # Remember to implement this!
```

When an external condition is triggered, the pass statement allows you to handle the condition without the loop being impacted in any way; all of the code will continue to be read unless a break or other statement occurs.

Using the same code block as above, let’s replace the break or continue statement with a pass statement:
```python
number = 0

for number in range(10):
    number = number + 1

    if number == 5:
        pass    # pass here

    print('Number is ' + str(number))

print('Out of loop')
```
The pass statement occurring after the if conditional statement is telling the program to continue to run the loop and ignore the fact that the variable number evaluates as equivalent to 5 during one of its iterations.

We’ll run the program and take a look at the output:
```
Number is 1
Number is 2
Number is 3
Number is 4
Number is 5
Number is 6
Number is 7
Number is 8
Number is 9
Number is 10
Out of loop
```
By using the pass statement in this program, we notice that the program runs exactly as it would if there were no conditional statement in the program. The pass statement tells the program to disregard that condition and continue to run the program as usual.

The pass statement can create minimal classes, or act as a placeholder when working on new code and thinking on an algorithmic level before hammering out details.

# Else in a Loop
Loop statements may have an else clause; it is executed when the loop terminates through exhaustion of the list (with for) or when the condition becomes false (with while), but not when the loop is terminated by a break statement.

Let's see the same code block as above:
```python
number = 0

for number in range(10):
    number = number + 1

    if number == 5:
        break    # continue here

    print('Number is ' + str(number))
else:
    print("I have already count to 10!")
    
print('Out of loop')
```
After we run the program and look at the result, we can find that the else statement is not executed since the loop is terminated by a break statement, failing to iterate over the list range(10).
```
Number is 1
Number is 2
Number is 3
Number is 4
Out of loop
```
If we change the if condition for the break statement as the following:
```python
number = 0

for number in range(10):
    number = number + 1

    if number == 15:
        break    # continue here

    print('Number is ' + str(number))
else:
    print("I have already count to 10!")
    
print('Out of loop')
```
The output becomes:
```
Number is 1
Number is 2
Number is 3
Number is 4
Number is 5
Number is 6
Number is 7
Number is 8
Number is 9
Number is 10
I have already count to 10!
Out of loop
```
Or, we can change like this
```python
number = 0

for number in range(10):
    number = number + 1

    if number == 5:
        continue    # continue here

    print('Number is ' + str(number))
else:
    print("I have already count to 10!")
print('Out of loop')
```
with the output
```
Number is 1
Number is 2
Number is 3
Number is 4
Number is 6
Number is 7
Number is 8
Number is 9
Number is 10
I have already count to 10!
Out of loop
```
We can conclude that as long as the loop can iterave over all of the iterations defined by the loop list -- range(10), then the else statement will be executed.

This can also be exemplified by the following loop, which searches for prime numbers:
```python
for n in range(2, 10):
    for x in range(2, n):
        if n % x == 0:
            print(n, 'is not a prime number, since', n, 'equals', x, '*', n//x)
            break
    else:
        # loop fell through without finding a factor
        print(n, 'is a prime number')
```
The output of the above code is
```
2 is a prime number
3 is a prime number
4 is not a prime number, since 4 equals 2 * 2
5 is a prime number
6 is not a prime number, since 6 equals 2 * 3
7 is a prime number
8 is not a prime number, since 8 equals 2 * 4
9 is not a prime number, since 9 equals 3 * 3
```
(Yes, this is the correct code. Look closely: the else clause belongs to the for loop, not the if statement.)

When used with a loop, the else clause has more in common with the else clause of a try statement than it does that of if statements: a try statement’s else clause runs when no exception occurs, and a loop’s else clause runs when no break occurs.

### 9.1.5 Applications


<font color='red'>Critical Points: </font>
1. What is each stage? How many stages will you have?
2. What are possible decisions for each stage?
3. What is the compatible condition?

<font color='red' size=6>Template: </font>
```python

def backtracking(results, solution, current_position, N, possible_decisions):
    """
    Question: 1. What is each stage? How many stages will you have?
    
    results: 最终返回结果，比如上例中的蛋糕总食谱
    solution: 当前stage的结果
    current_position: 当前的阶段stage（当前是第几个阶段）. It starts at 0.
    N: 总阶段数，比如上例中有3个阶段
    possible_decisions: 当前阶段可以做的决策种类
    """
    
    if len(solution) == N:
        # we have successfully build one answer here
        answers.append(solution[:])  # deep copy
        ......
    else:
        # Question: 2. What are possible decisions for each stage?
        for decision in possible_decisions[current_position]:
            # Question: 3. What is the compatible condition?
            if is_compatible(solution, decision):
                solution.add(decision)
                backtracking(results, solution, current_position+1, N, possible_decisions)
                solution.remove(decision)
```

#### Question 1: [Leetcode 51] [N-Queens](https://leetcode.com/problems/n-queens/)
The n-queens puzzle is the problem of placing n queens on an n×n chessboard such that no two queens attack each other.

![N Queens](source/lesson10_DFS_nqueens.png)

<img src="source/lesson10_DFS_nqueens_2.png", width=300>

Given an integer n, return all distinct solutions to the n-queens puzzle.

Each solution contains a distinct board configuration of the n-queens' placement, where 'Q' and '.' both indicate a queen and an empty space respectively.

Example:
```
Input: 4
Output: [
 [".Q..",  // Solution 1
  "...Q",
  "Q...",
  "..Q."],

 ["..Q.",  // Solution 2
  "Q...",
  "...Q",
  ".Q.."]
]
Explanation: There exist two distinct solutions to the 4-queens puzzle as shown above.
```



Note: Why cannot use for loop to deal with this problem?  
对应于每一行，我们要用一个嵌套的 for loop 遍历该行的每一列。对于这个问题，我们要用n个嵌套的 for loop。但是阶段数 n 是个变量，这导致我们不可能用 for loop 来解决这个问题。


Pseudo Code:
```python
def bt(results, positions, row, n):
    """
    Question: 1. What is each stage? How many stages will you have?
    Ans: Stage: row 1, row 2, ..., row n. We have n stages
    Ex:    Stage 1      Stage 2      Stage 3      Stage 4
            col 1        col 1        col 1        col 1
            col 2        col 2        col 2        col 2
            col 3        col 3        col 3        col 3
            col 4        col 4        col 4        col 4
    restuls: store all the possible valid chess configs
    positions[i]: the column of the queen that is placed at the ith row
    row: represents the current row we are trying to place the queen at
    n: the total number of rows
    """
    # Base case: find a valid config
    if len(positions) == n:
        results.append(...)
        return
    # For the current row, try all the possible columns to place our queen
    # Question: 2. What are possible decisions for each stage?
    # Ans: For each row, we can pick up a col from col 0, col 1, ..., col n
    for column in range(n):
        # Question: 3. What is the compatible condition?
        # Ans: the picked col in the current row should not be attacked by queens in previous rows
        #   that is, the picked col in current row is not equal to the occupied cols in previous rows
        #   and abs(the picked col - the previous col) is not equal to (the current row - the previous row)
        if is_compatible(positions, row, column, n):
            # if not conflict with all the previous queens
            positions.append(column)
            bt(results, positions, row+1, n)
            positions.pop()
```

Analysis: 
0. Assume the latest positions [x, x, x, Y1]
1. append(column) --> positions: [x, x, x, Y1, Y2]
2. pop() --> positions: [x, x, x, Y1]

Key Observations:
1. All queens will be on different rows.
2. We could find result by putting queens row by row (essentially, we could build result incrementally).
3. Based on the backtracking framework, assuming we have put queens to previous k rows and the ith queen on ith row is put in ci column, when we consider the k+1 row, we need to make sure the queen we will put in this row will not be on the same columns and diagonals as those previous ones.
4. Testing of potential collision on columns is easy. For diagonal testing, assuming right now the board is 4X4, we could encode each cell as the following. The domain for $c_{i}$ {1,2,...,n} and $P_{k}(c_1, c_2, \ldots, c_k)$ should be $c_{i} \neq c_{j}$ and $|c_{j}-c_{i}| \neq j-i$ for all $1 \leq i < j \leq k$

| col0| col1| col2| col3|
| --- | --- | --- | --- |
| 0,0 | 0,1 | 0,2 | 0,3 |
| 1,0 | 1,1 | 1,2 | 1,3 |
| 2,0 | 2,1 | 2,2 | 2,3 |
| 3,0 | 3,1 | 3,2 | 3,3 |

N皇后问题有个技巧的关键在于棋盘的表示方法，这里使用一个数组就可以表达了。比如 colns=[1, 3, 0, 2]，这是4皇后问题的一个解，意思是：在第0行，皇后放在第1列；在第1行，皇后放在第3列；在第2行，皇后放在第0列；在第3行，皇后放在第2列。

Time Complexity: worst case $O(n^n)$ in theory (we have n stage, each stage have n possible decisions). But in practice, it is much more efficient, since it will stop searching via one branch if the compatible condition is not satisfied.

In [3]:
class Solution:
    def solveNQueens(self, n):
        """
        :type n: int
        :rtype: List[List[str]]
        """
        results, solution = [], []
        # self.count = 0        
        
        self.backtracking(results, solution, 0, n)  # Bug Free 1: use self.fun()
        return results # , self.count
    
    def backtracking(self, results, solution, row, n):  # Bug Free 2: add self in the parameter list
        # Base case
        if row == n:
            # result = []
            # for col in solution:
            #     result.append('.' * col + 'Q' + '.' * (n - col - 1))  # Bug Free 3: use 'Q' instead of Q
            result = ['.' * col + 'Q' + '.' * (n-col-1) for col in solution]
            results.append(result)
            # self.count = self.count + 1
            return            
        
        # Recursion Body
        for col in range(n):
            if self.is_compatible(solution, row, col):
                solution.append(col)
                self.backtracking(results, solution, row + 1, n)
                solution.pop()  # Bug Free 4: to pop the last element is pop(), not pop(col)
                
                
    def is_compatible(self, solution, cur_row, cur_col):
        for pre_row in range(cur_row):
            # to avoid column collision and diagnosis collision
            pre_col = solution[pre_row]
            if pre_col == cur_col or abs(cur_col - pre_col) == cur_row - pre_row:
                return False
        return True
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.solveNQueens(n=4))

[['.Q..', '...Q', 'Q...', '..Q.'], ['..Q.', 'Q...', '...Q', '.Q..']]


#### Question 2: [Leetcode 46 Medium] [Permutations](https://leetcode.com/problems/permutations/)
Given a collection of distinct integers, return all possible permutations.

Note: the solution set must not contain duplicate subsets.

Example:
```
Input: [4,5,6]
Output:
[
  [4,5,6],
  [4,6,5],
  [5,4,6],
  [5,6,4],
  [6,4,5],
  [6,5,4]
]
```

*Solution 1*: 

Pseudo Code:
```python
def bt(results, perm, index, nums):
    """
    Question: 1. What is each stage? How many stages will you have?
    Ans: Stage: no. 1, no. 2, no. 3, ..., no. n. We have n stages for the total n numbers
    Ex:    Stage 1      Stage 2      Stage 3
              4            4            4
              5            5            5
              6            6            6
    restuls: store all the possible valid chess configs
    perm: store the current partial permutation that we have so far
    indx: represents the current position that we want to fill
    nums: a collection of distinct integers
    """
    # Base case: find a valid perm
    if len(positions) == len(nums):
        results.append(...)
        return
    # For all the possible num that we can pick for the current index
    # Question: 2. What are possible decisions for each stage?
    # Ans: For each stage, we can pick up a num from nums
    for num in nums:  # possible decisions for the current stage
        # Question: 3. What is the compatible condition?
        # Ans: the current num that we pick should not be used before
        # this is a simple rule, and hence we do not need an extra function to realize it
        # if is_compatible(perm, num):
        if num not in perm:
            # if not conflict with all the previous queens
            perm.append(num)
            bt(results, perm, index+1, nums)
            perm.pop()
```

key observations:
1. We can build a valid permutation incrementally.
2. Based on the input and our framework, the domain for each choice we will make when we consider a position in our permutations is all integers in our input and the condition for our partially built permutation is that: 
$$p_{i} \neq p_{j} \text{ for all } i \leq < j \leq k$$

In [9]:
class Solution:
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        Time Complexity: O(n!)
        Space Complexity: O(n)
        """
        results, solution = [], []
        
        self.backtracking(results, solution, 0, nums)
        
        return results
    
    def backtracking(self, results, solution, stage, nums):  # Bug Free 1: add self as the first parameter
        # base case
        if stage == len(nums):
            results.append(solution[:])  # Bug Free 2: solution[:]. Caputre the currnt object held by solution
            return
        
        # recursion body
        for num in nums:
            if num not in solution:
                
                solution.append(num)
                print(solution)
                print(stage+1)
                print('---')
                self.backtracking(results, solution, stage + 1, nums)
                solution.pop()
                print('sol', solution)
                    
if __name__ == "__main__":
    soln = Solution()
    #print(soln.permute(nums=[1,2,3]))
    print(soln.permute(nums=[4,5,6]))

[4]
1
---
[4, 5]
2
---
[4, 5, 6]
3
---
sol [4, 5]
sol [4]
[4, 6]
2
---
[4, 6, 5]
3
---
sol [4, 6]
sol [4]
sol []
[5]
1
---
[5, 4]
2
---
[5, 4, 6]
3
---
sol [5, 4]
sol [5]
[5, 6]
2
---
[5, 6, 4]
3
---
sol [5, 6]
sol [5]
sol []
[6]
1
---
[6, 4]
2
---
[6, 4, 5]
3
---
sol [6, 4]
sol [6]
[6, 5]
2
---
[6, 5, 4]
3
---
sol [6, 5]
sol [6]
sol []
[[4, 5, 6], [4, 6, 5], [5, 4, 6], [5, 6, 4], [6, 4, 5], [6, 5, 4]]


*Solution 2*: 

As you might notice, the time complexity for testing whether the current candidate we have picked or not is $O(n)$. In our case, it is easy to reduce this to $O(1)$. You either use an additional hash table to record the value we have picked, or we could reuse the input list itself to be the collection of all candidates. For the second approaches, basically, during the recusion, this list can be partitioned into two parts: elements we have picked for our partially built permutation and the remaining undertermined elements.

```
                     1 2 3
            /          |          \
        1 2 3        2 1 3        3 2 1
      /     \       /    \        /     \
  1 2 3   1 3 2   2 1 3  2 3 1  3 2 1  3 1 2
```

In [3]:
class Solution(object):
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        Time Complexity: O(n!)
        Space Complexity: O(n)
        """
        results, solution = [], []
        
        self.backtracking(results, solution=nums, idx=0)
        
        return results
    
    def backtracking(self, results, solution, idx):  # Bug Free 1: add self as the first parameter
        if idx == len(solution):
            results.append(solution[:])  # Bug Free 2: solution[:]. Caputre the currnt object held by solution
            return
        else:
            for i in range(idx, len(solution)):
                solution[idx], solution[i] = solution[i], solution[idx]
                self.backtracking(results, solution, idx + 1)
                solution[idx], solution[i] = solution[i], solution[idx]
                
if __name__ == "__main__":
    soln = Solution()
    print(soln.permute(nums=[1,2,3]))
    print(soln.permute(nums=[4,5,6]))

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 2, 1], [3, 1, 2]]
[[4, 5, 6], [4, 6, 5], [5, 4, 6], [5, 6, 4], [6, 5, 4], [6, 4, 5]]


*Solution 3: Divide and Conquer*

This problem can also be solved by divide and conquer strategy. In order to solve the problem of generating all permutations of $n$ distinctive elements, assuming we have already solved the smaller instances of the original problem: generating all permutations of n-1 distinctive elements. How do we combine these results? Let's look at one small example:

We want to get all permutations of sequence {1,2,3,4}. Assuming we know how to get the result for {2,3,4}. Then for every permutation in the resulting collection of {2,3,4}, if we prepend 1 to it, we will get a subset of all permutations of {1,2,3,4}. Similarly, for every permutation in the resulting collecion of {1,3,4}, if we prepend 2 to it, we will also get a subset of all permutations. One important thing to notice here is that these two subsets are independent of each other since all permutation in the first subset will start with 1 while the others start with 2.

Hence, we can have the following recursion rule based on this observation:  
Perm($x_{1}, x_{2}, x_{3}, \ldots, x_{n}$) = {prepend $x_{1}$ to permutation in Perm($x_{2}, x_{3}, \ldots, x_{n}$)} $\cup$ {prepend $x_{2}$ to permutation in Perm($x_{1}, x_{3}, \ldots, x_{n}$)} $\cup\ \ldots\ \cup$ {prepend $x_{n}$ to permutation in Perm($x_{1}, x_{2}, \ldots, x_{n-1}$)}

In [9]:
class Solution:
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        if not nums:
            return [[]]
        
        results = []
        for num in nums:
            # what to get from your children
            perms = self.permute(nums[:nums.index(num)] + nums[nums.index(num) + 1:])
            # print(perms)
            # what to do in the current stage
            for perm in perms:
                results.append([num] + perm)
                
        # what to return to your parent
        return results
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.permute(nums=[1,2,3]))
    print(soln.permute(nums=[4,5,6]))

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]
[[4, 5, 6], [4, 6, 5], [5, 4, 6], [5, 6, 4], [6, 4, 5], [6, 5, 4]]


In [10]:
class Solution:
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        if not nums:
            return [[]]
        
        return [[num] + perm for num in nums for perm in 
                self.permute(nums[:nums.index(num)] + nums[nums.index(num)+1:])]
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.permute(nums=[1,2,3]))
    print(soln.permute(nums=[4,5,6]))

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]
[[4, 5, 6], [4, 6, 5], [5, 4, 6], [5, 6, 4], [6, 4, 5], [6, 5, 4]]


Alternatively, with {1,2,3,4} still as our example, after getting all permutations of {2,3,4}, instead of prepend every such permutation with 1, we can consider, for example for a given permutation {3,2,4}, how many different ways we could turn it back to a valid permutation of {1,2,3,4} by adding the missing 1 back?

There are 4 ways: {1,3,2,4}, {3,1,2,4}, {3,2,1,4}, and {3,2,4,1}. Thus we have another recursion rule.
Perm($x_{1}, x_{2}, x_{3}, \ldots, x_{n}$) = {insert $x_{1}$ to permutation in Perm($x_{2}, x_{3}, \ldots, x_{n}$) at every possible positions}

In [11]:
class Solution:
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        if not nums:
            return [[]]
        
        return [ perm[:pos] + [nums[0]] + perm[pos:] for perm in 
                self.permute(nums[1:]) for pos in range(len(perm) + 1)]
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.permute(nums=[1,2,3]))
    print(soln.permute(nums=[4,5,6]))

[[1, 2, 3], [2, 1, 3], [2, 3, 1], [1, 3, 2], [3, 1, 2], [3, 2, 1]]
[[4, 5, 6], [5, 4, 6], [5, 6, 4], [4, 6, 5], [6, 4, 5], [6, 5, 4]]


If you notice the way we do computation in the above codes, basically, there is no backtracking and the order of the actual computation is from the smallest instance to the original instance of the problem. Thus, it is very easy to turn the above recursion code into an iterative code.

In [12]:
class Solution:
    def permute(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        res = [[]]
        for num in nums:
            res = [perm[:i] + [num] + perm[i:] for perm in res for i in range(len(perm)+1)]
        
        return res
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.permute(nums=[1,2,3]))
    print(soln.permute(nums=[4,5,6]))

[[3, 2, 1], [2, 3, 1], [2, 1, 3], [3, 1, 2], [1, 3, 2], [1, 2, 3]]
[[6, 5, 4], [5, 6, 4], [5, 4, 6], [6, 4, 5], [4, 6, 5], [4, 5, 6]]


#### Question 3: [Leetcode 78 Medium] [Subsets](https://leetcode.com/problems/subsets/)
Given a set of distinct integers, nums, return all possible subsets (the power set).

Note: The solution set must not contain duplicate subsets.

Example:
```
Input: [1,2,3]
Output:
[
  [3],
  [1],
  [2],
  [1,2,3],
  [1,3],
  [2,3],
  [1,2],
  []
]
```


[1,2,3] --> n=3 stages decisions problem
* Stage 1: pick 1 or not pick 1
* Stage 2: pick 2 or not pick 2
* Stage 3: pick 3 or not pick 3

Analysis:  
DFS Basic Approach
1. What does it store on each level? (每层代表什么意义？一般来讲解题之前就知道DFS要recurse多少层。)  
Three levels. For each level, it makes the decision on whether to put this element into the final set.
2. How many different states should we try to put on this level? (每层有多少个状态/case需要try?)  
Two. Each state (case) considers either select or not selected.

![DFS subsets](source/lesson10_DFS_subset.png)

Pseudo Code:
```python
def bt(all_possible_subsets, subset, stage_index, nums):
    """
    Question: 1. What is each stage? How many stages will you have?
    Ans: Stage: no. 1, no. 2, no. 3, ..., no. n. We have n stages for the total n numbers
    Ex:    Stage 1      Stage 2      Stage 3
            Y 1          Y 2          Y 3
            N 1          N 2          N 3
    """
    if stage_index == len(nums):
        ...
        return
    else:
        # Question: 2. What are possible decisions for each stage?
        # Ans: For each stage, we can choose to pick up or not pick up the num 
        # There are only two decisions to make: pick nums[index] or not
        # Question: 3. What is the compatible condition?
        # Ans: always true
        # Case 1: not pick
        bt(all_possible_subsets, subset, stage_index+1, nums)
        # Case 2: pick
        subset.append(nums[stage_index])
        bt(all_possible_subsets, subset, stage_index+1, nums)
        # Move back, prepare for the next iteration
        subset.pop()
```

Key observations:
1. We can build a valid subset incrementally.
2. One key difference between this problem and the permutation problem is that, at a certain position in our candidate set, we are not forced to always choose an item. Omitting it is also a valid option.
3. Based on the input and our framework as well as point 2, when we consider a candidate, we should only consider the ones haven't been considered before. If elements are deliberately being omitted in previous selections, these are also counted as considerations.

In [13]:
class Solution:
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        Time Complexity: O(2^n)
        """
        results, solution = [], []
        
        self.backtracking(results, solution, 0, nums)
        return results
    
    def backtracking(self, results, solution, stage_index, nums):
        if stage_index == len(nums):
            results.append(solution[:])
            return
        
        # Case 1: pick up the num
        solution.append(nums[stage_index])
        self.backtracking(results, solution, stage_index+1, nums)            
        # remove the num from the solution; recover
        solution.pop()

        # Case 2: do not pick up the num
        self.backtracking(results, solution, stage_index+1, nums)    
            
if __name__ == "__main__":
    soln = Solution()
    print(soln.subsets(nums=[1,2,3]))

[[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]


*Solution: Divide and Conquer*:

We want to get all subsets of sequence {1,2,3}. Assuming we know how to get the result for {1,2}. Based on the definition of subsets, we know that this result is part of the final result of {1,2,3}. The previous result essentailly contains all subsets that do not contain 3. Then the remaining result that is independent of this one is those that do contain 3:
Subset($x_{1}, x_{2}, x_{3}, \ldots, x_{n}$) = Subset($x_{1}, x_{2}, x_{3}, \ldots, x_{n-1}$) $\cup$ {add $x_{n}$ to every subset in Subset($x_{1}, x_{2}, x_{3}, \ldots, x_{n-1}$)}

In [16]:
class Solution:
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        if not nums:
            return [[]]
        
        # what to get from your children
        child_res = self.subsets(nums[:-1])
        
        # what to do in the current stage
        curr_res = []
        for subset in child_res:
            curr_res.append(subset + [nums[-1]])
            
        # what to return to your parent
        return child_res + curr_res
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.subsets(nums=[1,2,3]))

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]


In [64]:
class Solution:
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        if not nums:
            return [[]]
        
        res = self.subsets(nums[:-1])
        
        return res + [subset + [nums[-1]] for subset in res]
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.subsets(nums=[1,2,3]))

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]


By similar reasoning, we could have the following iterative codes:

In [17]:
class Solution:
    def subsets(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        res = [[]]
        
        for num in nums:
            for i in range(len(res)):
                res.append(res[i] + [num])
                
        return res
    
if __name__ == "__main__":
    soln = Solution()
    print(soln.subsets(nums=[1,2,3]))

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]


#### Question 3*: [Leetcode 90 Medium] [Subsets II](https://leetcode.com/problems/subsets-ii/)
Given a collection of integers that might contain duplicates, nums, return all possible subsets (the power set).

Note: The solution set must not contain duplicate subsets.

Example:
```
Input: [1,2,2]
Output:
[
  [2],
  [1],
  [1,2,2],
  [2,2],
  [1,2],
  []
]
```

```
a x b x b
0 x 0 x 0 = 8 combination
1   1   1
```
we can solve in this way,
```
a x 2b
0 x 0 = 6 combination
1   1
    2
```

For more complicated example, e.g., string = "a b b b c", we have
```
a x 3b x c
0 x 0  x 0 = 16 combination
1   1    1
    2
    3
```
![DFS subsets](source/lesson10_DFS_subset3.png)

```
 a b1 b2 b3 c
          |
        index
[], [ab1], [ab2](wrong, we cannot add b2 if there is no b1 ahead)
[ab1b2], [ab1b3](wrong, we cannot add b3 if there is no b1 or b2 ahead)
...
```

In [3]:
class Solution(object):
    def subsetsWithDup(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        Time Complexity: O(2^n)
        Space Complexity: O(n)
        """
        if not nums:
            return []
        
        results, solution = [], []
        
        self.backtracking(results, solution, 0, nums)
        return results
    
    def backtracking(self, results, solution, stage_idx, nums):
        # base case
        if stage_idx == len(nums):
            results.append(solution[:])
            return
        
        # recursion rules
        # case 1: add nums[idx] into solution
        solution.append(nums[stage_idx])
        self.backtracking(results, solution, stage_idx + 1, nums)        
        # recover the state
        solution.pop()
        
        # a b1 b2 b3 c
        #   i       (i)
        i = stage_idx
        cur_num = nums[stage_idx]
        while i < len(nums) and nums[i] == cur_num:
            i += 1
        # case 2: add nothing
        self.backtracking(results, solution, i, nums)
        
if __name__ == "__main__":
    soln = Solution()
    print(soln.subsetsWithDup(nums=[1,2,2]))

[[1, 2, 2], [1, 2], [1], [2, 2], [2], []]


#### Question 4: [Leetcode 22 Medium] [Generate Parentheses](https://leetcode.com/problems/generate-parentheses/)
Given n pairs of parentheses, write a function to generate all combinations of well-formed parentheses.

For example, given n = 3, a solution set is:
```
[
  "((()))",
  "(()())",
  "(())()",
  "()(())",
  "()()()"
]
```

**Backtracking for fine grained control**  
We could build the partial parentheses sequence incrementally. Based on the definition of a valid parentheses sequence, if we have a ')' in our partially built sequence that cannot be matched to any of the preceding unmatched '(', there is no point of continuing building our solution. This suggests that the number of remaining '(' should always <= the number of remaining ')'. If this holds, when we add those ')''s, we can still find a matching '(''s.

Thus:
1. For every selection, the candidates will be either '(' or ')'
2. For the condition, we should only add '(' when some still remains. In order to keep the aforementioned invariant between the number of remaining '(' and the number of remaining ')', we should only add ')' when the first number is less than the second one.

Pseudo Code:    
```python
def bt(all_valid_seqs, seq, l, r, n):
    """
    Question: 1. What is each stage? How many stages will you have?
    Ans: Stage: no. 1, no. 2, no. 3, ..., no. n. We have n stages for the total n numbers
    Ex:    Stage 1      Stage 2      Stage 3
            left         left         left
            right        right        right
    """
    # l: the remaining number of left parenthesis
    # r: the ramaining number of right parenthesis  
    
    # base case := we have made all the decisions
    if len(seq) == 2 * n:
        ......
        return
    
    # Question: 2. What are possible decisions for each stage?
    # Ans: For each stage, we can choose to add '(' or ')'
    
    # Question: 3. What is the compatible condition?
    # Ans: (1) '(' can have the maximum number of 3
    #      (2) ')' can be placed only if the previous # '(' is larger than # ')'
        
    # Case 1: try to add '('
    if l > 0:
        seq.append('(')
        bt(all_valid_seqs, seq, l-1, r, n)
        seq.pop()
    # Case 2: try to add ')'
    # invariant: l <= r --> guarantee valid sequence
    if l < r:
        seq.append(')')
        bt(all_valid_seqs, seq, l, r-1, n)
        seq.pop()
```


Analysis:  
DFS Basic Approach
1. What does it store on each level? (每层代表什么意义？一般来讲解题之前就知道DFS要recurse多少层。)  
选择加左括号或者右括号
2. How many different states should we try to put on this level? (每层有多少个状态/case需要try?)  
2

产生n的左括号和n个右括号  
什么时候能加左括号，现在加的左括号的数量 < n；  
什么时候能加右括号，现在加的左括号的数量 > 现在加的右括号的数量

Time Complexity: $O(2^{2n})$

In [19]:
class Solution:
    def generateParenthesis(self, n):
        """
        :type n: int
        :rtype: List[str]
        """
        results, solution = [], []
        
        self.backtracking(results, solution, 0, 0, n)
        return results
    
    def backtracking(self, results, solution, left, right, n):
        if len(solution) == 2*n:  # or we can use if right == 3
            results.append("".join(solution[:]))
            return

        # Case 1: add '('
        if left < 3:
            solution.append('(')
            self.backtracking(results, solution, left+1, right, n)
            solution.pop()
        # Case 2: add ')'
        if right < left:
            solution.append(')')
            self.backtracking(results, solution, left, right+1, n)
            solution.pop()
                
if __name__ == "__main__":
    soln = Solution()
    print(soln.generateParenthesis(n=3))

['((()))', '(()())', '(())()', '()(())', '()()()']


*Solution: Divide and Conquer* (Not really ^_^)

Based on the pattern, assuming S, A, and B, are all valid parentheses sequence. Then the following two are true:
1. (S) is a valid parentheses sequence.
2. The concatenation of A and B is also a valid parentheses sequence.

Thus, we could have the following recursion rule:  
Generate(n) = {(S) for every in Generate(n-1)} $\cup$ {cartesian product of Generate(1) and Generate(n-1)} $\cup$ {cartesian product of Generate(2) and Generate(n-2)} $\cup$ ... $\cup$ {cartesian product of Generate(n-1) and Generate(1)}

Notice that, except the first one, all the remaining subsets are not independent of each other. For example, when n = 4, "()(())()" is included in both {cartesian product of Generate(1) and Generate(3)} and {cartesian product of Generate(3) and Generate(1)}. Thus when implementing the combine part of those results from the samller instances, we need to deduplicate.

In [20]:
def impl(n):
    if n == 0:
        return set([''])
        
    return set(['()' + s + ')' for s in impl(n-1)]) | set([s1 + s2 for k in range(1,n) for s1 in impl(k) for s2 in impl(n-k)])

class Solution:
    def generateParenthesis(self, n):
        """
        :type n: int
        :rtype: List[str]
        """
        return list(impl(n))
        
if __name__ == "__main__":
    soln = Solution()
    print(soln.generateParenthesis(n=3))

['())())())', '())()()))', '()())()))', '()()))())', '()()())))']


#### Question 4*: Follow up (Advanced):  

What if we have 3 kinds of parentheses ((), <>, {})  
Idea: we need to keep track of last unmatched left parenthese  
Use a stack

In [22]:
class Solution:
    def generateParenthesis(self, l, m, n):
        """
        :type n: input: int l, int m, int n
        :rtype: List[str]
        """
        results, solution, stack = [], [], []
        stage_index = [0, 0, 0, 0, 0, 0]
        total_index = [l, l, m, m, n, n]
        parentheses = ['(', ')', '<', '>', '{', '}']
        
        self.backtracking(results, solution, stack, stage_index, total_index, parentheses)
        return results
    
    def backtracking(self, results, solution, stack, stage_index, total_index, parentheses):
        if len(solution) == sum(total_index):  # or we can use if right == 3
            results.append("".join(solution[:]))
            return
        
        for i in range(len(stage_index)):
            if i % 2 == 0: # Case 1: add left parentheses
                if stage_index[i] < total_index[i]:
                    solution.append(parentheses[i])
                    stack.append(parentheses[i])
                    stage_index[i] += 1
                    self.backtracking(results, solution, stack, stage_index, total_index, parentheses)
                    # recover state
                    solution.pop()
                    stack.pop()
                    stage_index[i] -=1
            else:  # Case 2: add right parentheses
                if stack and stack[-1] == parentheses[i-1]:
                    solution.append(parentheses[i])
                    stack.pop()
                    stage_index[i] += 1
                    self.backtracking(results, solution, stack, stage_index, total_index, parentheses)
                    # recover state
                    solution.pop()
                    stack.append(parentheses[i-1])  # Bug Free: add back the left parenthese, the current one is right
                    stage_index[i] -=1

if __name__ == "__main__":
    soln = Solution()
    print(soln.generateParenthesis(l=1, m=1, n=1))

['()<>{}', '()<{}>', '(){<>}', '(){}<>', '(<>){}', '(<>{})', '(<{}>)', '({<>})', '({})<>', '({}<>)', '<()>{}', '<(){}>', '<({})>', '<>(){}', '<>({})', '<>{()}', '<>{}()', '<{()}>', '<{}()>', '<{}>()', '{()<>}', '{()}<>', '{(<>)}', '{<()>}', '{<>()}', '{<>}()', '{}()<>', '{}(<>)', '{}<()>', '{}<>()']


#### Question 4**: Follow up (Advanced):  

What if we have 3 kinds of parentheses ((), <>, {}) and want to enforce priority {}  >  <>  > ()  
Use a stack

In [19]:
class Solution:
    def generateParenthesis(self, l, m, n):
        """
        :type n: input: int l, int m, int n
        :rtype: List[str]
        """
        results, solution, stack = [], [], []
        stage_index = [0, 0, 0, 0, 0, 0]
        total_index = [l, l, m, m, n, n]
        parentheses = ['(', ')', '<', '>', '{', '}']
        
        self.backtracking(results, solution, stack, stage_index, total_index, parentheses)
        return results
    
    def backtracking(self, results, solution, stack, stage_index, total_index, parentheses):
        if len(solution) == sum(total_index):  # or we can use if right == 3
            results.append("".join(solution[:]))
            return
        
        for i in range(len(stage_index)):
            if i % 2 == 0: # Case 1: add left parentheses
                if stage_index[i] < total_index[i]:
                    if stack and self.get_priority(stack[-1]) < self.get_priority(parentheses[i]):
                        continue
                    solution.append(parentheses[i])
                    stack.append(parentheses[i])
                    stage_index[i] += 1
                    self.backtracking(results, solution, stack, stage_index, total_index, parentheses)
                    # recover state
                    solution.pop()
                    stack.pop()
                    stage_index[i] -=1
            else:  # Case 2: add right parentheses
                if stack and stack[-1] == parentheses[i-1]:
                    solution.append(parentheses[i])
                    stack.pop()
                    stage_index[i] += 1
                    self.backtracking(results, solution, stack, stage_index, total_index, parentheses)
                    # recover state
                    solution.pop()
                    stack.append(parentheses[i-1])  # Bug Free: add back the left parenthese, the current one is right
                    stage_index[i] -=1

    def get_priority(self, left_parent):
        if left_parent == '{':
            return 2
        elif left_parent == '<':
            return 1
        else:
            return 0
                    
if __name__ == "__main__":
    soln = Solution()
    print(soln.generateParenthesis(l=1, m=1, n=1))

['()<>{}', '(){<>}', '(){}<>', '<()>{}', '<>(){}', '<>{()}', '<>{}()', '{()<>}', '{()}<>', '{<()>}', '{<>()}', '{<>}()', '{}()<>', '{}<()>', '{}<>()']


#### Question 5: [Leetcode 79 Medium] [Word Search](https://leetcode.com/problems/word-search/)
(for student who is ok with the previous 4 problems)  
Given a 2D board and a word, find if the word exists in the grid.

The word can be constructed from letters of sequentially adjacent cell, where "adjacent" cells are those horizontally or vertically neighboring. The same letter cell may not be used more than once.

Example:
```
board =
[
  ['A','B','C','E'],
  ['S','F','C','S'],
  ['A','D','E','E']
]

Given word = "ABCCED", return true.
Given word = "SEE", return true.
Given word = "ABCB", return false.
```


Key observation:
1. Can build the desired path in the grid incrementally.
2. For every selection, the candidates will be the 4 adjacent cells around the current cell.
3. For the condition, we only want to proceed if the next cell we haven't visited before and it mathches the next character in the target word.

<font color='red'>Critical Points: </font>
1. What is each stage? How many stages will you have?  
   Ans: Stage: no. 1, no. 2, no. 3, ..., no. n, n = len(word) We have n stages for the word.  
   ```
   Ex:    Stage 1 ('S')     Stage 2 ('E')     Stage 3 ('E')
           left              left              left
           right             right             right
           up                up                up
           down              down              down
   ```
2. What are possible decisions for each stage?  
   Ans: For each stage, we can move up, down, left or right. We have four possible decisions in total.
3. What is the compatible condition?  
   Ans: Assume the current position is (x, y) and the next/new position is (nx, ny)
  * borad[x][y] == word[stage_idx] (or we can use borad[nx][ny] == word[stage_idx+1])
  * I should stay within the range of the borad. 0 <= nx < len(borad) and 0 <= ny < len(board[0])
  * The new position should not be visited again. For example, we cannot move up and then move down to visit the same letter cell. "The same letter cell may not be used more than once."

In [1]:
class Solution:
    def exist(self, board, word):
        """
        :type board: List[List[str]]
        :type word: str
        :rtype: bool
        """
        if not board or not board[0] or not word:
            return False
        
        for x in range(len(board)):
            for y in range(len(board[0])):
                if board[x][y] == word[0]:
                    visited = set()
                    #solution = [word[0]]
                    if self.backtracking(x, y, board, 0, word, visited):
                        return True
        
        return False
    
    def backtracking(self, x, y, board, stage_idx, word, visited):
        # base case
        if stage_idx == len(word) - 1:
            return True

        dx, dy = [1, 0, -1, 0], [0, -1, 0, 1]
        direction = ['down', 'left', 'up', 'right']
        
        visited.add((x,y))           

        for direct in range(4): # up, down, left, right
            nx = x + dx[direct]
            ny = y + dy[direct]

            if ((0 <= nx < len(board)) and (0 <= ny < len(board[0])) and 
            ((nx, ny) not in visited) and (board[nx][ny] == word[stage_idx+1])):

                if self.backtracking(nx, ny, board, stage_idx+1, word, visited):  # Bug Free: remember to use new x and new y
                    return True                    

        visited.discard((x, y))      
        return False
    
if __name__ == "__main__":
    board = [
      ['A','B','C','E'],
      ['S','F','C','S'],
      ['A','D','E','E']
    ]
    soln = Solution()
    print(soln.exist(board, word="ABCCED"))
    print(soln.exist(board, word="SEE"))
    print(soln.exist(board, word="ABCB"))

True
True
False


#### Question 6: [Leetcode 200 Medium] [Number of Islands](https://leetcode.com/problems/number-of-islands/)

Given a 2d grid map of '1's (land) and '0's (water), count the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

Example 1:
```
Input:
11110
11010
11000
00000

Output: 1
```
Example 2:
```
Input:
11000
11000
00100
00011

Output: 3
```

[*Solution*](https://zxi.mytechroad.com/blog/searching/leetcode-200-number-of-islands/)

In [20]:
class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        if not grid or not grid[0]:
            return 0
        
        count = 0
        
        visited = [[False for col in range(len(grid[0]))] for row in range(len(grid))]
        # Bug Free: we should not use the following initialization for visited
        # Such initialization will lead to one column points to the same box
        # visited = len(grid) * [len(grid[0]) * [False]] 
                
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                if grid[row][col] == 1 and not visited[row][col]:
                    visited[row][col] = True
                    count = count + 1
                    self.backtracking(row, col, grid, visited)
                    
        return count
                    
    def backtracking(self, row, col, grid, visited):
        # base case
        if row == len(grid) or col == len(grid[0]):
            return
        
        drow = [-1, 0, 1, 0]
        dcol = [0, -1, 0, 1]
        
        for d in range(4):
            new_row = row + drow[d]
            new_col = col + dcol[d]
            
            if (0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and 
               grid[new_row][new_col] == 1 and not visited[new_row][new_col]):
                #print((new_row, new_col), grid[new_row][new_col])
                visited[new_row][new_col] = True
                #print(visited)
                self.backtracking(new_row, new_col, grid, visited)
                
if __name__ == "__main__":
    grid = [[1,1,1,1,0], 
        [1,1,0,1,0], 
        [1,1,0,0,0],
        [0,0,0,0,0]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,0,0], 
            [1,1,0,0,0], 
            [0,0,1,0,0],
            [0,0,0,1,1]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,1,1], 
            [1,1,0,0,1], 
            [0,0,1,0,0],
            [0,0,0,0,1]]
    soln = Solution()
    print(soln.numIslands(grid))

1
3
4


In [5]:
class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        if not grid or not grid[0]:
            return 0
        
        count = 0
        
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                if grid[row][col] == 1:
                    grid[row][col] == 0
                    count = count + 1
                    self.backtracking(row, col, grid)
                    
        return count
                    
    def backtracking(self, row, col, grid):
        # base case
        if row == len(grid) or col == len(grid[0]):
            return
        
        drow = [-1, 0, 1, 0]
        dcol = [0, -1, 0, 1]
        
        for d in range(4):
            new_row = row + drow[d]
            new_col = col + dcol[d]
            
            if (0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and 
               grid[new_row][new_col] == 1):
                grid[new_row][new_col] = 0
                self.backtracking(new_row, new_col, grid)   
                
if __name__ == "__main__":
    grid = [[1,1,1,1,0], 
        [1,1,0,1,0], 
        [1,1,0,0,0],
        [0,0,0,0,0]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,0,0], 
            [1,1,0,0,0], 
            [0,0,1,0,0],
            [0,0,0,1,1]]
    soln = Solution()
    print(soln.numIslands(grid))

    grid = [[1,1,0,1,1], 
            [1,1,0,0,1], 
            [0,0,1,0,0],
            [0,0,0,0,1]]
    soln = Solution()
    print(soln.numIslands(grid))

1
3
4


#### Question 7: [Leetcode 518] [Coin Change 2](https://leetcode.com/problems/coin-change-2/)
Print all combinations of coins that can sum up to a total value k.

You are given coins of different denominations and a total amount of money. Write a function to compute the number of combinations that make up that amount. You may assume that you have infinite number of each kind of coin. 

Example 1:
```
Input: amount = 5, coins = [1, 2, 5]
Output: 4
Explanation: there are four ways to make up the amount:
5=5
5=2+2+1
5=2+1+1+1
5=1+1+1+1+1
```
Example 2:
```
Input: amount = 3, coins = [2]
Output: 0
Explanation: the amount of 3 cannot be made up just with coins of 2.
```
Example 3:
```
Input: amount = 10, coins = [10] 
Output: 1
```
Example:
```
Input: amount = 99, coins = [25, 10, 5, 1] 
Output: 213
```

*Analysis*:  
  
DFS Basic Approach
1. What does it store on each level? (每层代表什么意义？一般来讲解题之前就知道DFS要recurse多少层。)  
Four levels (len(coints)). For each level, we consider one coin value. 每一层加一种硬币
2. How many different states should we try to put on this level? (每层有多少个状态/case需要try?)  
加0,1,2，...,n-1(len(range(n)), where n = amount // coin + 1)(当前还需要钱/当前硬币的币值)

```
                                                       root(99)
                                  /                 /           \               \
level1 (25 cent)       0*25(remain 99)          1*25(74)       2*25(49)        3*25(24)     
                     /      /      |   \           |             |        /       |        \     
level2 (10 cent) 0*10(99) 1*10(89) ... 9*10(9)    ...           ...    0*10(24) 1*10(14) 2*10(4)
                  /    \                                                                    |     
level3 (5 cent) 0*5(99) ...                                                               0*5(4)
                  |                                                                         |      
level4 (1 cent)  ...                                                                      4*1(0)  
```
Time Complexity: T = O(99^4)

```
Base case: #已经组成了target value
    print(solution)
Recursion rules:
    for i in range(remaining values + 1):
        solution.append(i)
        dfs(coins, index+1, solution)
        solution.pop()
```

In [6]:
# return the results

class Solution(object):
    def change(self, amount, coins):
        """
        :type amount: int
        :type coins: List[int]
        :rtype: int
        Time Complexity: O(target^(len(coins))) = branch number ^ level, worst case: (target/min(coins))^n
        Space Complexity: n
        """
        if amount < 0 or not coins:
            return []
        
        results, solution = [], []
        self.back_tracking(results, solution, 0, amount, coins)
        return results
    
    def back_tracking(self, results, solution, stage_index, amount, coins):
        if stage_index == len(coins):
            if amount == 0:
                results.append(solution[:])                
            return
        
        coin = coins[stage_index]
        for i in range(0, amount // coin + 1):
            solution.append(i)
            self.back_tracking(results, solution, stage_index + 1, amount - i*coin, coins)
            solution.pop()
            
if __name__ == "__main__":
    soln = Solution()

    print(soln.change(amount=10, coins=[5,2,1]))
    print(soln.change(amount=5, coins=[5,2]))

[[0, 0, 10], [0, 1, 8], [0, 2, 6], [0, 3, 4], [0, 4, 2], [0, 5, 0], [1, 0, 5], [1, 1, 3], [1, 2, 1], [2, 0, 0]]
[[1, 0]]


In [8]:
# return the number of choices

class Solution(object):
    def change(self, amount, coins):
        """
        :type amount: int
        :type coins: List[int]
        :rtype: int
        """
        if amount < 0 or not coins:
            return 0
        
        self.count = 0

        self.back_tracking(0, amount, coins)
        return self.count
    
    def back_tracking(self, stage_index, amount, coins):
        if stage_index == len(coins):
            if amount == 0:
                self.count += 1             
            return
        
        coin = coins[stage_index]
        for i in range(0, amount // coin + 1):
            self.back_tracking(stage_index + 1, amount - i*coin, coins)
    
if __name__ == "__main__":
    soln = Solution()

    print(soln.change(amount=99, coins=[25,10,5,1]))
    print(soln.change(amount=10, coins=[5,2,1]))
    print(soln.change(amount=5, coins=[5,2]))

213
10
1


#### Question 8: [Leetcode 254 Medium] [Factor Combinations](https://leetcode.com/problems/factor-combinations/)
Numbers can be regarded as product of its factors. For example,
```
8 = 2 x 2 x 2;
  = 2 x 4.
  
12 = 2 x 2 x 3
   = 2 x 6
   = 3 x 4
   
```
Write a function that takes an integer n and return all possible combinations of its factors.

Note: 
1. Each combination's factors must be sorted ascending, for example: The factors of 2 and 6 is [2, 6], not [6, 2].
2. You may assume that n is always positive.
3. Factors should be greater than 1 and less than n.
 
Examples: 
```
input: 1
output: 
[]

input: 37
output: 
[]

input: 12
output:
[
  [2, 6],
  [2, 2, 3],
  [3, 4]
]

input: 32
output:
[
  [2, 16],
  [2, 2, 8],
  [2, 2, 2, 4],
  [2, 2, 2, 2, 2],
  [2, 4, 4],
  [4, 8]
]
```

*Solution 1*:  
Analysis:  
DFS Basic Approach
1. What does it store on each level? (每层代表什么意义？一般来讲解题之前就知道DFS要recurse多少层。)  
the number of factors
2. How many different states should we try to put on this level? (每层有多少个状态/case需要try?)  
dynamic: the number of divisors

In [11]:
import math

class Solution(object):
    def get_factor(self, n):
        """
        :type n: int
        :rtype: List[List[int]]
        """
        if n <= 1:
            return []

        results, solution = [], []

        factors = self.get_valid_factors(n)
        self.back_tracking(results, solution, 0, n, factors)
        
        return results
    
    def back_tracking(self, results, solution, stage_idx, n, factors):
        if stage_idx == len(factors):
            if n == 1:
                results.append(self.convert(factors, solution))
            return
        
        factor = factors[stage_idx]
        for i in range(0, math.floor(math.log(n, factor)+1)):
            if n % (factor ** i) == 0:
                solution.append(i)
                self.back_tracking(results, solution, stage_idx+1, n // (factor ** i), factors)
                solution.pop()
    
    
    def get_valid_factors(self, n):
        factors = []
        for i in range(2, n//2+1):
            if n % i == 0:
                factors.append(i)
        return factors
    
    def convert(self, factors, cur):
        res = []
        for i in range(len(factors)):
            num = cur[i]
            while num > 0:
                res.append(factors[i])
                num -= 1
        return res
    
if __name__ == "__main__":
    soln = Solution()

    print(soln.get_factor(n=1))
    print(soln.get_factor(n=37))
    print(soln.get_factor(n=12))
    print(soln.get_factor(n=32))

[]
[]
[[3, 4], [2, 6], [2, 2, 3]]
[[4, 8], [2, 16], [2, 4, 4], [2, 2, 8], [2, 2, 2, 4], [2, 2, 2, 2, 2]]


Preprocess all factors of 12: 6 4 3 2.            O(n) <<<< DFS's Time Complexity  

*Solution 2*:  
Analysis:  
DFS Basic Approach
1. What does it store on each level? (每层代表什么意义？一般来讲解题之前就知道DFS要recurse多少层。)  
the max number of divisors
2. How many different states should we try to put on this level? (每层有多少个状态/case需要try?)  
the number of factors

```
                                                       root(12)
                                  /                 /           \               \
level0 (first factor)      2(remain 6)             3(4)         4(3)            6(2)     
                            /        \              |                                          
level1 (second factor)    2(3)       3(2)          2(2)          
...
```
Time Complexity: T = O(factor^logn)  
Space Complexity: S = O(logn)

In [12]:
class Solution(object):
    def get_factor(self, n):
        """
        :type n: int
        :rtype: List[List[int]]
        """
        if n <= 1:
            return []

        results, solution = [], []

        self.back_tracking(results, solution, 2, n)
        results.pop()
        return results
    
    def back_tracking(self, results, solution, factor, num):
        if num == 1:
            results.append(solution[:])
            return
        
        for new_factor in range(factor, num + 1):
            if num % new_factor == 0:
                solution.append(new_factor)
                self.back_tracking(results, solution, new_factor, num//new_factor)
                solution.pop()
                
if __name__ == "__main__":
    soln = Solution()

    print(soln.get_factor(n=1))
    print(soln.get_factor(n=37))
    print(soln.get_factor(n=12))
    print(soln.get_factor(n=32))

[]
[]
[[2, 2, 3], [2, 6], [3, 4]]
[[2, 2, 2, 2, 2], [2, 2, 2, 4], [2, 2, 8], [2, 4, 4], [2, 16], [4, 8]]


#### Question 4: [Leetcode 22] Generate Parentheses
Given n pairs of parentheses, write a function to generate all combinations of well-formed parentheses.

For example, given n = 3, a solution set is:
```
[
  "((()))",
  "(()())",
  "(())()",
  "()(())",
  "()()()"
]
```

Analysis:  
DFS Basic Approach
1. What does it store on each level? (每层代表什么意义？一般来讲解题之前就知道DFS要recurse多少层。)  
选择加左括号或者右括号
2. How many different states should we try to put on this level? (每层有多少个状态/case需要try?)  
2

产生n的左括号和n个右括号  
什么时候能加左括号，现在加的左括号的数量 < n；  
什么时候能加右括号，现在加的左括号的数量 > 现在加的右括号的数量

Time Complexity: $O(2^{2n})$

In [69]:
def generate_parentheses(n):
    cur = []
    res = []
    dfs(0, 0, n, cur, res)
    return res

def dfs(left, right, n, cur, res):
    if left == n and right == n:
        res.append(''.join(cur))
        return
    if left < n:
        cur.append('(')
        dfs(left+1, right, n, cur, res)
        cur.pop()  # recover curr state
    if right < left:
        cur.append(')')
        dfs(left, right+1, n, cur, res)
        cur.pop()
        
# Time Complexity: O(2^n)
# Space: O(n)

In [68]:
print(generate_parentheses(n=3))

['((()))', '(()())', '(())()', '()(())', '()()()']


Follow up (Advanced):  (no need to master)  
What if we have 3 kinds of parentheses ((), <>, {})  
Idea: we need to keep track of last unmatched left parenthese  
Use a stack

In [70]:
def generate_parentheses(l, m, n):
    """
    input: int l, int m, int n
    return: string[]
    """
    remain = [l, l, m, m, n, n]
    parentheses = ['(', ')', '<', '>', '{', '}']
    target_len = 2 * (l + m + n)
    cur = []
    stack = []
    res = []
    dfs(cur, stack, remain, target_len, res, parentheses)
    return res

def dfs(cur, stack, remain, target_len, res, parentheses):
    if len(cur) == target_len:
        res.append(''.join(cur))
        return
    for i in range(len(parentheses)):
        if i % 2 == 0:  # add left parentheses
            if remain[i] > 0:
                cur.append(parentheses[i])
                stack.append(parentheses[i])
                remain[i] -= 1
                dfs(cur, stack, remain, target_len, res, parentheses)
                # recover state
                cur.pop()
                stack.pop()
                remain[i] += 1
        else:
            if stack and stack[-1] == parentheses[i-1]:
                cur.append(parentheses[i])
                stack.pop()
                remain[i] -= 1
                dfs(cur, stack, remain, target_len, res, parentheses)
                # recover state
                cur.pop()
                stack.append(parentheses[i-1])
                remain[i] += 1
        
# Time Complexity: O(6^2(l+m+n))
# Space: O(l+m+n)

In [72]:
print(generate_parentheses(l=1, m=1, n=1))

['()<>{}', '()<{}>', '(){<>}', '(){}<>', '(<>){}', '(<>{})', '(<{}>)', '({<>})', '({})<>', '({}<>)', '<()>{}', '<(){}>', '<({})>', '<>(){}', '<>({})', '<>{()}', '<>{}()', '<{()}>', '<{}()>', '<{}>()', '{()<>}', '{()}<>', '{(<>)}', '{<()>}', '{<>()}', '{<>}()', '{}()<>', '{}(<>)', '{}<()>', '{}<>()']


Follow up:  (no need to master)    
What if we have 3 kinds of parentheses ((), <>, {}) and want to enforce priority {}  >  <>  > ()  
Use a stack

In [39]:
def generate_parentheses(l, m, n):
    """
    input: int l, int m, int n
    return: string[]
    """
    remain = [l, l, m, m, n, n]
    parentheses = ['(', ')', '<', '>', '{', '}']
    target_len = 2 * (l + m + n)
    cur = []
    stack = []
    res = []
    dfs(cur, stack, remain, target_len, res, parentheses)
    return res

def dfs(cur, stack, remain, target_len, res, parentheses):
    if len(cur) == target_len:
        res.append(''.join(cur))
        return
    for i in range(len(parentheses)):
        if i % 2 == 0:  # add left parentheses
            if remain[i] > 0:
                if stack and get_priority(stack[-1]) < get_priority(parentheses[i]):
                    continue
                cur.append(parentheses[i])
                stack.append(parentheses[i])
                remain[i] -= 1
                dfs(cur, stack, remain, target_len, res, parentheses)
                # recover state
                cur.pop()
                stack.pop()
                remain[i] += 1
        else:
            if stack and stack[-1] == parentheses[i-1]:
                cur.append(parentheses[i])
                stack.pop()
                remain[i] -= 1
                dfs(cur, stack, remain, target_len, res, parentheses)
                # recover state
                cur.pop()
                stack.append(parentheses[i-1])
                remain[i] += 1

def get_priority(left_parent):
    if left_parent == '{':
        return 2
    elif left_parent == '<':
        return 1
    else:
        return 0
                
# Time Complexity: O(6^2(l+m+n))
# Space: O(l+m+n)

In [74]:
print(generate_parentheses(l=1, m=1, n=1))

['()<>{}', '(){<>}', '(){}<>', '<()>{}', '<>(){}', '<>{()}', '<>{}()', '{()<>}', '{()}<>', '{<()>}', '{<>()}', '{<>}()', '{}()<>', '{}<()>', '{}<>()']


## 9.2 Leetcode Training (Basic)

[Leetcode 0017 Medium] [Letter Combinations of a Phone Number](Leetcode_0017.ipynb) (Backtracking)

[Leetcode 0022 Medium] [Generate Parentheses](Leetcode_0022.ipynb) (Backtracking)

[Leetcode 0079 Medium] [Word Search](Leetcode_0079.ipynb) (Backtracking)  
[Leetcode 0212 Hard] [Word Search II](Leetcode_0212.ipynb) (Backtracking)

## 9.3 Leetcode Practice (Advanced)